In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' # Make matplotlib render retina-quality charts
import seaborn as sns
from IPython.display import display

## 1. First look at the data

In [ ]:
data = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Some values in TotalCharges were not able to load as numeric.
# I manually handle it by replacing them by the product of tenure and monthly charges.
data.TotalCharges = pd.to_numeric(data.TotalCharges, errors='coerce')
data.loc[data.TotalCharges.isnull(), 'TotalCharges'] = data.tenure * data.MonthlyCharges

print("Original dataset sample:")
display(data.head(5))
display(data.info())

### What do I notice immediately:
- The quality of data is really good. It was clear from Kaggle's preview, and now confirmed. There are no missing values, and all columns have meaningful sense and values.
- "customerID" column contains unique IDs (confirmed at Kaggle's preview), so we can delete it.
- Since a lot of feature are multi-level categorical features, we will need to do a lot of encoding.

## 2. Feature-by feature analysis
**Important**: if I decide to drop or encode some features, I will consecutively add their names to special lists initialized in the cell below, and delete/encode them in 1 line in the end of the feature-by-feature analysis.

In [ ]:
columns_to_delete = [] # Here we will accumulate names of columns for deletion
columns_to_encode = [] # Here we will accumulate names of columns for encoding

def plot_crosstab_and_value_counts_againts_target(categorical_feature):
    """This function takes a name of a categorical feature as string,
    and plots crosstab and value_counts of it versus the target."""
    
    print("Count of values: {} versus Churn:".format(categorical_feature))
    display(pd.crosstab(data.Churn, data[categorical_feature], margins=True).style.background_gradient(cmap='summer_r'))
    print("Average churn rate for each {}:".format(categorical_feature))
    display(data.groupby(categorical_feature)['Churn'].mean().sort_values(ascending=False))
    
def display_correlation_with_binned_version(numerical_feature):
    """This function takes a name of a numerical feature as string,
    and plots crosstab and value_counts of it 5-binned version versus the target."""
    
    feature_binned_name = numerical_feature + "_binned"
    data[feature_binned_name] = pd.cut(data[numerical_feature], bins = 5)
    plot_crosstab_and_value_counts_againts_target(feature_binned_name)
    data.drop(feature_binned_name, axis = 1, inplace = True)

### (Target) Churn
This column is binary, and contains string values "Yes" and "No". Let's encode it to boolean at the beginning, because it will allow us to calculate average churn rates per different dimensions of data.

In [ ]:
data.Churn = data.Churn.map({"Yes": 1, "No": 0})

### (1/20) customerID
As was indicate above, this feature can be deleted.

In [ ]:
columns_to_delete.append("customerID")

### (2/20) gender – whether the customer is a male or a female

In [ ]:
plot_crosstab_and_value_counts_againts_target('gender')
columns_to_delete.append("gender")

**Decision**: this feature alone seems to have no correlation with the target. **Don't include** in the models.

### (3/20) SeniorCitizen – whether the customer is a senior citizen or not (1, 0)

In [ ]:
plot_crosstab_and_value_counts_againts_target('SeniorCitizen')

**Decision**: for both senior and non-senior citizens, the churn rate is <50%. However, for senior citizens it is 2 times higher than for non-senior! We will include this feature in the model. **Include** in the models.

### (4/20) Partner – whether the customer has a partner or not

In [ ]:
plot_crosstab_and_value_counts_againts_target('Partner')
columns_to_encode.append('Partner')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (5/20) Dependents – whether the customer has dependents or not (Yes, No)

In [ ]:
plot_crosstab_and_value_counts_againts_target('Dependents')
columns_to_encode.append('Dependents')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (6/20) tenure – number of months the customer has stayed with the company

In [ ]:
display_correlation_with_binned_version('tenure')

**Decision**: **Include** in the models. We see clear negative correlation between the tenure and the churn rate.

### (7/20) PhoneService – whether the customer has a phone service or not (Yes, No)

In [ ]:
plot_crosstab_and_value_counts_againts_target('PhoneService')
columns_to_delete.append('PhoneService')

**Decision**: this feature alone seems to have no correlation with the target. **Don't include** in the models.

### (8/20) MultipleLines – whether the customer has multiple lines or not (Yes, No, No phone service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('MultipleLines')
columns_to_delete.append('MultipleLines')

**Decision**: this feature alone seems to have no correlation with the target. **Don't include** in the models.

### (9/20) InternetService – customer’s internet service provider (DSL, Fiber optic, No)

In [ ]:
plot_crosstab_and_value_counts_againts_target('InternetService')
columns_to_encode.append('InternetService')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (10/20) OnlineSecurity – whether the customer has online security or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('OnlineSecurity')
columns_to_encode.append('OnlineSecurity')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (11/20) OnlineBackup – whether the customer has online backup or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('OnlineBackup')
columns_to_encode.append('OnlineBackup')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (12/20) DeviceProtection – whether the customer has device protection or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('DeviceProtection')
columns_to_encode.append('DeviceProtection')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (13/20) TechSupport – whether the customer has tech support or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('TechSupport')
columns_to_encode.append('TechSupport')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (14/20) StreamingTV – whether the customer has streaming TV or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('StreamingTV')
columns_to_delete.append('StreamingTV')

**Decision**: this feature alone seems to have no correlation with the target. **Don't include** in the models.

### (15/20) StreamingMovies – whether the customer has streaming movies or not (Yes, No, No internet service)

In [ ]:
plot_crosstab_and_value_counts_againts_target('StreamingMovies')
columns_to_delete.append('StreamingMovies')

**Decision**: this feature alone seems to have no correlation with the target. **Don't include** in the models.

### (16/20) Contract – the contract term of the customer (Month-to-month, One year, Two year)

In [ ]:
plot_crosstab_and_value_counts_againts_target('Contract')
columns_to_encode.append('Contract')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (17/20) PaperlessBilling – whether the customer has paperless billing or not (Yes, No)

In [ ]:
plot_crosstab_and_value_counts_againts_target('PaperlessBilling')
columns_to_encode.append('PaperlessBilling')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (18/20) PaymentMethod – the customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))

In [ ]:
plot_crosstab_and_value_counts_againts_target('PaymentMethod')
columns_to_encode.append('PaymentMethod')

**Decision**: **Include** in the models. We see the big difference in average churn rate between the category classes.

### (19/20) MonthlyCharges – the amount charged to the customer monthly

In [ ]:
display_correlation_with_binned_version('MonthlyCharges')

**Decision**: **Include** in the models. We see clear positive correlation between the monthly charges and churn rate.

### (20/20) TotalCharges – the total amount charged to the customer

In [ ]:
display_correlation_with_binned_version('TotalCharges')

**Decision**: **Include** in the models. We see clear negative correlation between the total charges and churn rate.

## 3. Preprocessing of data

In [ ]:
print("We decided to delete columns: " + str(columns_to_delete))
print("We decided to encode columns: " + str(columns_to_encode))

"""
6 columns tell us similar information – whether a customer has a specific service connected {Yes, No, No internet at all}.
However, we have a distinct feature (InternetService) to consider those who don't have internet, so to avoid
multicollinearity, we need to join "No" and "No internet service" in all six columns.
"""
columns_with_no_and_ns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                          'TechSupport', 'StreamingTV', 'StreamingMovies']

for column in columns_with_no_and_ns:
    data.loc[:,column].replace("No internet service", "No", inplace = True)
    
# Next, we have 2 categories in PaymentMethod with almost identical churn rate. Let's merge them.
data.PaymentMethod = data.PaymentMethod.map({
    'Electronic check': 'Electronic check',
    'Mailed check': 'Mailed check',
    'Bank transfer (automatic)': "Automatic",
    'Credit card (automatic)': "Automatic",
})

# Let's generate a new feature - total number of online services connected.
data['NumberOnlineServices'] = np.where(data.OnlineSecurity == 'Yes', 1, 0) + \
                               np.where(data.OnlineBackup == 'Yes', 1, 0) + \
                               np.where(data.DeviceProtection == 'Yes', 1, 0) + \
                               np.where(data.TechSupport == 'Yes', 1, 0) + \
                               np.where(data.StreamingTV == 'Yes', 1, 0) + \
                               np.where(data.StreamingMovies == 'Yes', 1, 0)
                               
plot_crosstab_and_value_counts_againts_target('NumberOnlineServices')

print("Starting the deletion and encoding, as requested...")
# Here we drop and encode all columns which we marked for deletion or encoding.
data.drop(columns_to_delete, axis = 1, inplace = True)
data = pd.get_dummies(data,
                      columns = columns_to_encode,
                      drop_first = True
                      )
print("Deletion and encoding finished")

# Separate y and X
y = data.Churn
X = data.drop('Churn', axis = 1)

## 4. Creating 5 alternative models
Most ML models have **hyperparameters** – parameters that should be specified before fitting the model. The process of choosing optimal hyperparameters is called **hyperparameters tuning**, and that's exactly what I do here.

Let me introduce another concept – **regularization**. Regularization is the process of making a model more able to catch general trends instead of train-specific trends, by modifying the loss function. 'C' hyperparameter in logistic regression (and in many other models) is responsible for it. Low C-values lead to lower train score and higher test score, because the model becomes able to catch general trends better. However, excessively low c-values will make a model not able to catch any trends at all.

For each of the five models, my approach would be:
1. Compare models with different key hyperparameter's values.
1. Select the optimal value of key hyperparameter, based on highest cross-validated precision.
1. Compare the best versions of each of the 5 models with each other.

Also, before running it, I'd like to specify that in this project my **target metric is precision**.
**Precision** in general is the fraction of relevant instances among the retrieved instances. In this problem, it is a fraction of clients who did churn among those which were selected by the model as likely to chyrn. Precision reaches its best value at 1 and worst score at 0.

I'm choosing precision for 2 reasons:
- To provide business value for telecoms, it is better for this model to have lower recall but higher precision than vise versa. Predicting too much churn would mean losing money on ads which try to get people back while they don't plan to leave. On the other hand, we know that conversion rates of ads targeted to people who really want to churn are quite low, so we should spend them only for those who are really likely to churn.
- We are facing a problem of imbalanced classes, and accuracy makes no sense in such problems.

In [ ]:
# Import functions which will be used in all models
from sklearn.model_selection import cross_val_score, ShuffleSplit
# Create a split object – it will be used in all models.
split = ShuffleSplit(n_splits=3, test_size=0.25, random_state=1111)

### Model 1: Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define an empty list to store precision for different hyperparameters
lr_precisions = list()

# Values of C to be checked
lr_C_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]

# Loop over C values to find an optimal
for C_value in lr_C_values:
    
    # Create LogisticRegression object
    lr_foropt = LogisticRegression(solver = 'liblinear', penalty = 'l1', C = C_value, max_iter=1000)
    
    # Fit it to the cross validator
    lr_cv_results_list = cross_val_score(lr_foropt, X, y, cv = split, scoring = 'precision')
    
    # Compute the mean precision
    lr_avg_precision = np.mean(lr_cv_results_list)
    
    # Append this mean to the list of precisions
    lr_precisions.append(lr_avg_precision)
    
# Plot the results
plt.plot(lr_C_values, lr_precisions)
plt.title('C-value optimization')
plt.ylabel('Cross-validated precision score')
plt.xlabel('C value')
plt.show()

# Print the value of C-hyperparameter which was found optimal
lr_optimalC = lr_C_values[lr_precisions.index(max(lr_precisions))]
print('Optimal c-value: ', lr_optimalC)

# Create logistic regression with our optimal hyperparameter
lr = LogisticRegression(solver = 'liblinear', penalty = 'l1', C = lr_optimalC, max_iter=1000)

### Model 2: Decision Tree Classifier
In this classificator I'll tune 2 hyperparameters: criterion and max_depth of the tree. The instrument for tuning more that 1 hyperparameter is called **Grid Search**, and I apply it here:

In [ ]:
# Import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Initialize a standard classifier
dt_foropt = DecisionTreeClassifier()

# Import GridSearchCV package
from sklearn.model_selection import GridSearchCV

# Instantiate the GridSearchCV object and run the search
parameters_dt = {'criterion':['gini', 'entropy'],
                 'max_depth': np.arange(3,100,2)}

searcher_dt = GridSearchCV(dt_foropt, parameters_dt, cv = split, scoring = 'precision')
searcher_dt.fit(X, y)

# Report the best parameters and the corresponding score
print("Best CV params", searcher_dt.best_params_)

# Initialize a classifier with optimal parameters
dt = DecisionTreeClassifier(criterion = searcher_dt.best_params_['criterion'],
                            max_depth = searcher_dt.best_params_['max_depth'],
                            class_weight = 'balanced')

### Model 3: KNN Classifier
The key hyperparameter of KNN is K – the number of neighbors to consider in voting.

In [ ]:
# Import KNN from scikit learn
from sklearn.neighbors import KNeighborsClassifier

# Define an empty list to store precision for different hyperparameters
knn_precisions = list()

# Values of K to be checked
neighbors = np.arange(1,50,2)

# Loop over K values to find an optimal
for k in neighbors:
    
    # Create KNeighborsClassifier object
    knn_foropt = KNeighborsClassifier(n_neighbors = k)
    
    # Fit it to the cross validator
    knn_cv_results_list = cross_val_score(knn_foropt, X, y, cv = split, scoring = 'precision')
    
    # Compute the mean precision
    knn_avg_precision = np.mean(knn_cv_results_list)
    
    # Append this mean to the list of precision scores
    knn_precisions.append(knn_avg_precision)
    
# Plot the results
plt.plot(neighbors, knn_precisions)
plt.title('K value optimization')
plt.ylabel('Cross-validated precision')
plt.xlabel('Neighbors')
plt.show()

# Print the value of K-hyperparameter which was found optimal
knn_optimal_k = neighbors[knn_precisions.index(max(knn_precisions))]
print('Optimal number of neighbors: ', knn_optimal_k)

# Create a model with our optimal hyperparameter
knn = KNeighborsClassifier(n_neighbors = knn_optimal_k)

### Model 4: CatBoost Classifier
CatBoost Classifier is based on decision trees. It's key hyperparameter, according to the documentation, is the regularization strength, called "l2_leaf_reg".

In [ ]:
from catboost import CatBoostClassifier

# Define an empty list to store precision for different hyperparameters
cbr_precisions = list()

# Values of l2_leaf_reg to be checked
l2_values = [0.01, 0.1, 0.25, 0.5, 1, 2, 3, 5, 10, 20, 50, 100, 200]

# Loop over l2_leaf_reg values to find an optimal
for l2 in l2_values:
    
    # Create CatBoostClassifier object
    cbr_foropt = CatBoostClassifier(l2_leaf_reg = l2, silent = True)
    
    # Fit it to the cross validator
    cv_results_list_cbr = cross_val_score(cbr_foropt, X, y, cv = split, scoring = 'precision')
    
    # Compute the mean precision
    cbr_avg_precision = np.mean(cv_results_list_cbr)
    
    # Append this mean to the list of precisions
    cbr_precisions.append(cbr_avg_precision)
    
# Plot the results
plt.plot(l2_values, cbr_precisions)
plt.title('CBR optimization')
plt.ylabel('Precision')
plt.xlabel('l2_leaf_reg')
plt.show()

# Print the value of l2-hyperparameter which was found optimal
optimal_l2 = l2_values[cbr_precisions.index(max(cbr_precisions))]
print('Optimal l2-value: ', optimal_l2)

# Create CatBoostClassifier with our optimal hyperparameter
cbr = CatBoostClassifier(l2_leaf_reg = optimal_l2, silent = True)

### Model 5. AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators = 100)

## 5. Comparing models

In [ ]:
from sklearn.metrics import plot_confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score

def print_report_for_classifier(model, model_name = 'unnamed classifier'):
    """This function takes and UNtrained but created model object and its verbose name as parametets,
    and prints a report with its performance of the entire dataset.
    Please, note, that this report assumes that we test the model on the same data as we use to train it,
    because we've already tuned the hyperparameters in a way to aviod overfitting."""
    
    model.fit(X, y)
    y_pred = model.predict(X)
    
    # Calculate necessary data for ROC/AUC
    fpr, tpr, thresholds = roc_curve(y, y_pred)
    area = auc(fpr, tpr)

    print("Report for {}:".format(model_name))
    print('-'*100)
    
    # The next block of code just creates a figure with 2 charts
    fig = plt.figure(figsize = (8, 4), dpi = 100)

    axes11 = fig.add_subplot(121)
    plot_confusion_matrix(model, X, y, cmap = 'summer_r', values_format = "1", ax = axes11)
    axes11.set_title('Confusion matrix')

    axes12 = fig.add_subplot(122)
    axes12.plot(fpr, tpr, color = 'orange', label = "ROC curve (area = {:.2f})".format(area))
    axes12.plot([0,1], [0,1], linestyle='--', label = 'Diagonal')
    axes12.set_xlim([0,1])
    axes12.set_ylim([0,1.05])
    axes12.set_xlabel('False Positive Rate')
    axes12.set_ylabel('True Positive Rate')
    axes12.set_title('ROC')
    axes12.legend()

    plt.tight_layout()
    plt.show()
    print('-'*100)
    
    # We include 4 key metrics in our report
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    print('Accuracy: ', round(accuracy, 2))
    print('Precision: ', round(precision, 2))
    print('Recall: ', round(recall, 2))
    print('F1: ', round(f1, 2))
    print('AUC: ', round(area, 2))
    print('-'*100)

def display_comparison_table(models_with_names):

    column_names = ["Model", "Precision", "Recall", "F1", "Accuracy", "AUC"]
    df = pd.DataFrame(columns = column_names)

    for model, name in models_with_names:

        model.fit(X, y)
        y_pred = model.predict(X)
        fpr, tpr, thresholds = roc_curve(y, y_pred)

        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        f1 = f1_score(y, y_pred)
        accuracy = accuracy_score(y, y_pred)
        area = auc(fpr, tpr)

        df = df.append({
            "Model": name,
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "Accuracy": accuracy,
            "AUC": area
        }, ignore_index = True)

    def highlight_max_value_in_each_column(s):
        is_max = s == s.max()
        return ['background-color: PaleGreen' if v else '' for v in is_max]    

    df = df.set_index('Model').sort_values(by = 'Precision', ascending = False)

    display(
        df.style\
            .apply(highlight_max_value_in_each_column)\
            .set_precision(2)
    )

# Declare all models which we used
models_with_names = [
    (lr, 'Logistic Regression'),
    (dt, 'Decision Tree'),
    (knn, 'KNN classifier'),
    (cbr, 'CatBoost Classifier'),
    (ada, 'AdaBoost Classifier')
]    


display_comparison_table(models_with_names)    

In [ ]:
for model, name in models_with_names:
    print_report_for_classifier(model, name)

## 6. Making conclusions
According to our criteria – precision, the best model is **CatBoost Classifier**. It gives us not only the best precision, but the best values for accuracy, AUC and F1 score. The optimal l2_leaf_reg value is 100.